In [ ]:
## Experiments: Training the entire dense layer

In [1]:
from Executor import *
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("first").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [2]:
executor.load_precomputed_conv_models()
# Get the linear model 
linearModel = executor.get_rescaled_fc_model(new_dropout=executor.dropout)

# Set all layers except the last as being trainable
for layer in linearModel.layers:
    if layer.trainable:
        print("found trainable layer: ",layer)


loading precomputed conv. outputs...
done...
getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)
('found trainable layer: ', <keras.layers.pooling.MaxPooling2D object at 0x7fab67ba9ed0>)
('found trainable layer: ', <keras.layers.core.Flatten object at 0x7fab67b36310>)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7fab67b362d0>)
('found trainable layer: ', <keras.layers.core.Dropout object at 0x7fab67dce390>)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7fab6b1a0590>)
('found trainable layer: ', <keras.layers.core.Dropout object at 0x7fab6b1a0ed0>)
('found trainable layer: ', <ker

In [3]:
# QUITE BAD...
linearModel.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 8s - loss: 7.6935 - acc: 0.5114 - val_loss: 7.8770 - val_acc: 0.5113
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 7.7511 - acc: 0.5191 - val_loss: 7.8770 - val_acc: 0.5113
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 7.7511 - acc: 0.5191 - val_loss: 7.8770 - val_acc: 0.5113
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 7.7511 - acc: 0.5191 - val_loss: 7.8770 - val_acc: 0.5113
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 7.7511 - acc: 0.5191 - val_loss: 7.8770 - val_acc: 0.5113


In [4]:
# Now try to finetune the model, then maybe set the all dense layers trainable

In [5]:
# Get the linear model 
linearModel2 = executor.get_rescaled_fc_model(new_dropout=executor.dropout)

# Set all layers except the last as being trainable
i=0
for layer in linearModel2.layers:
    i+=1
    if(i!=7):
        layer.trainable=False

for layer in linearModel2.layers:
    if layer.trainable:
        print("found trainable layer: ",layer)


getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7faaa128b910>)


In [6]:
linearModel2.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel2.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 10, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/10
6255/6255 [==============================] - 3s - loss: 1.7487 - acc: 0.4566 - val_loss: 1.1659 - val_acc: 0.4855
Epoch 2/10
6255/6255 [==============================] - 3s - loss: 1.4904 - acc: 0.4708 - val_loss: 0.9618 - val_acc: 0.5294
Epoch 3/10
6255/6255 [==============================] - 3s - loss: 1.2940 - acc: 0.5014 - val_loss: 0.9304 - val_acc: 0.5571
Epoch 4/10
6255/6255 [==============================] - 3s - loss: 1.1773 - acc: 0.5084 - val_loss: 0.9329 - val_acc: 0.5694
Epoch 5/10
6255/6255 [==============================] - 3s - loss: 1.0969 - acc: 0.5322 - val_loss: 0.8857 - val_acc: 0.5726
Epoch 6/10
6255/6255 [==============================] - 3s - loss: 1.0944 - acc: 0.5161 - val_loss: 0.8996 - val_acc: 0.5591
Epoch 7/10
6255/6255 [==============================] - 3s - loss: 1.0245 - acc: 0.5423 - val_loss: 0.8777 - val_acc: 0.5842
Epoch 8/10
6255/6255 [==============================] - 3s - loss: 1.0274 - a

In [7]:
for layer in linearModel2.layers:
    layer.trainable = True

In [8]:
linearModel2.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel2.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 8s - loss: 0.8954 - acc: 0.5856 - val_loss: 0.8176 - val_acc: 0.6139
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.7520 - acc: 0.6521 - val_loss: 0.7804 - val_acc: 0.6340
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.6499 - acc: 0.7207 - val_loss: 0.7472 - val_acc: 0.6572
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.5539 - acc: 0.7640 - val_loss: 0.7171 - val_acc: 0.6669
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.4654 - acc: 0.8128 - val_loss: 0.7023 - val_acc: 0.6772


In [9]:
linearModel2.compile(optimizer=Adam(lr=0.000001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel2.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 8s - loss: 0.3980 - acc: 0.8488 - val_loss: 0.6996 - val_acc: 0.6753
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.3932 - acc: 0.8483 - val_loss: 0.6968 - val_acc: 0.6791
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.3634 - acc: 0.8611 - val_loss: 0.6945 - val_acc: 0.6804
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.3629 - acc: 0.8631 - val_loss: 0.6927 - val_acc: 0.6837
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.3484 - acc: 0.8705 - val_loss: 0.6903 - val_acc: 0.6850


In [10]:
executor.build_predictions_on_test_data().and_().save_predictions_to_file("foobar")

Found 512 images belonging to 1 classes.


<Executor.Executor instance at 0x7fab6b89ecb0>

In [11]:
executor.vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [12]:
executor.init_vgg_with_retrained_fc_layers(linearModel2)

<Executor.Executor instance at 0x7fab6b89ecb0>

In [13]:
executor.vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]   

In [14]:
executor.build_predictions_on_test_data().and_().save_predictions_to_file("foobar")

Found 512 images belonging to 1 classes.


<Executor.Executor instance at 0x7fab6b89ecb0>